In [ ]:
import numpy as np
import joblib
from langdetect import detect 
import fasttext  
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
# Cargamos el modelo
modelo_path = '../analysis/model3/modelo_multilingue.pkl'
objetos = joblib.load(modelo_path)

model = objetos['model']
le = objetos['label_encoder']
encoder = objetos['onehot_encoder']

In [ ]:
from gensim.models import KeyedVectors

# Cargar el modelo en ingles
fasttext_en = KeyedVectors.load_word2vec_format("../vectorizadores/wiki-news-300d-1M.bin", binary=True)

In [6]:
# Cargar modelos de embeddings
fasttext_es = fasttext.load_model("../vectorizadores/cc.es.300.bin")

In [ ]:
def predecir_sentimiento(texto):
    # Detectar idioma
    idioma = detect(texto)
    
    modelo_embedding = fasttext_es if idioma == 'es' else fasttext_en
    
    palabras = texto.split()
    
    if idioma == 'es': 
        vectores = [modelo_embedding.get_word_vector(p) for p in palabras if p]
    else:
        vectores = [modelo_embedding[p] for p in palabras if p in modelo_embedding]
    
    embedding = np.mean(vectores, axis=0) if vectores else np.zeros(300)
    idioma_encoded = encoder.transform([[idioma]])
    
    X_nuevo = np.hstack([embedding.reshape(1, -1), idioma_encoded])
    sentimiento_num = model.predict(X_nuevo)[0]
    
    return le.inverse_transform([sentimiento_num])[0]

In [ ]:
#ejemplo
texto_ejemplo = "ordering this is a waste of time. 7 delays from the provider. every time within an hour of delivery time. I've never been so annoyed with a pc company. It's amazing amazon will still provide a service for them. Great work MSI."
print("Sentimiento predicho:", predecir_sentimiento(texto_ejemplo))

Sentimiento predicho: enojado


c:\Users\ADMIN\Desktop\Programacion\python\FinalSic\feelings_enviorement\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
